# Autograd

## 计算图

计算图通常包含两种元素，一个是 tensor，另一个是 Function。

In [66]:
import torch

x  = torch.randn(2, requires_grad=True, device='cuda')
print(x)

tensor([ 0.4011, -0.9069], device='cuda:0', requires_grad=True)


In [67]:
x.tolist()

[0.40107426047325134, -0.906868040561676]

In [69]:
a = torch.tensor(2.0, requires_grad=True)
b = a.exp()
b

tensor(7.3891, grad_fn=<ExpBackward>)

在我们做正向传播的过程中，除了执行 forward() 操作之外，还会同时会为反向传播做一些准备，为反向计算图添加 Function 节点。在上边这个例子中，变量 b 在反向传播中所需要进行的操作是 `<ExpBackward>`。

<br />

## 一个具体的例子

**定义一个前向传播**

In [54]:
inp = torch.ones([2, 2], requires_grad=True)
w1 = torch.tensor(2.0, requires_grad=True)
w2 = torch.tensor(3.0, requires_grad=True)
w3 = torch.tensor(4.0, requires_grad=True)

l1 = inp * w1
l2 = l1 + w2
l3 = l1 * w3
l4 = l2 * l3
loss = l4.mean()

In [55]:
print(w1.data, w1.grad, w1.grad_fn)

tensor(2.) None None


In [57]:
print(l1.data, l1.grad, l1.grad_fn)

tensor([[2., 2.],
        [2., 2.]]) None <MulBackward0 object at 0x7f40edae4220>


<ipython-input-57-c589a7d06409>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(l1.data, l1.grad, l1.grad_fn)


In [58]:
print(loss.data, loss.grad, loss.grad_fn)

tensor(40.) None <MeanBackward0 object at 0x7f40edae4a00>


<ipython-input-58-e63e1720befd>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(loss.data, loss.grad, loss.grad_fn)


In [59]:
print(inp.grad)

None


**反向传播**

In [60]:
loss.backward()

In [61]:
print(w1.grad, w2.grad, w3.grad)

tensor(28.) tensor(8.) tensor(10.)


`w1` 等是叶子节点，导数被保留下来

In [62]:
print(l1.grad, l2.grad, l3.grad, l4.grad, loss.grad)

None None None None None


<ipython-input-62-793453e07697>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(l1.grad, l2.grad, l3.grad, l4.grad, loss.grad)


`l1` 等是非叶子节点，导数没有被保留下来

In [63]:
print(inp.grad)

tensor([[14., 14.],
        [14., 14.]])


**一些概念**

1. `tensor.is_leaf` 用来判断是否是叶子张量；
2. 在 BP 中，只有 `is_leaf = True` 的时候，需要求导的张量的导数结果才会被最后保留下来；
3. 由用户创建的 tensor 是一个叶子节点，而由其他运算操作产生的 tensor 则不是；
4. 将 `requires_grad = False` 的 tensor，约定俗成地归为叶子张量。

In [32]:
a = torch.ones([2, 2], requires_grad=False)
print(a.is_leaf)

True


**保留中间变量的导数**

In [22]:
input = torch.ones([2, 2], requires_grad=False)
w1 = torch.tensor(2.0, requires_grad=True)
w2 = torch.tensor(3.0, requires_grad=True)
w3 = torch.tensor(4.0, requires_grad=True)

l1 = input * w1
l2 = l1 + w2
l3 = l1 * w3
l4 = l2 * l3
loss = l4.mean()

# 使用 tensor.retain_grad() 保留中间变量的导数
l1.retain_grad()
l4.retain_grad()
loss.retain_grad()

loss.backward()

In [23]:
loss.grad

tensor(1.)

In [24]:
l4.grad

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])

In [25]:
l1.grad

tensor([[7., 7.],
        [7., 7.]])

如果只想 debug，而不需要保存中间变量的导数信息，可以使用 `tensor.register_hook` ([pytorch中的钩子（Hook）有何作用？](https://www.zhihu.com/question/61044004))

In [26]:
input = torch.ones([2, 2], requires_grad=False)
w1 = torch.tensor(2.0, requires_grad=True)
w2 = torch.tensor(3.0, requires_grad=True)
w3 = torch.tensor(4.0, requires_grad=True)

l1 = input * w1
l2 = l1 + w2
l3 = l1 * w3
l4 = l2 * l3
loss = l4.mean()

# 使用 tensor.register_hook
l1.register_hook(lambda grad: print('l1 grad: ', grad))
l4.register_hook(lambda grad: print('l4 grad: ', grad))
loss.register_hook(lambda grad: print('loss grad: ', grad))

loss.backward()

print(loss.grad)

loss grad:  tensor(1.)
l4 grad:  tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])
l1 grad:  tensor([[7., 7.],
        [7., 7.]])
None


<ipython-input-26-a2af1cf65e2c>:19: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(loss.grad)


loss 的 grad 在 print 完之后就被清除掉了

<br />

## inplace 操作

> 官方不推荐使用 inplace 的方式操作 tensor

In [15]:
A = np.array([[1,2,3],[4,5,6]])
result = A + [[2], [3]]
print(result)

[[3 4 5]
 [7 8 9]]


In [27]:
# 情景 1
a = torch.tensor([3.0, 1.0])
print(id(a))
a = a.exp()
print(id(a))
# 在这个过程中 a.exp() 生成了一个新的对象，然后再让 a
# 指向它的地址，所以这不是个 inplace 操作

139917664196864
139917664196736


In [28]:
# 情景 2
a = torch.tensor([3.0, 1.0])
print(id(a))
a[0] = 10
print(id(a), a)
# inplace 操作，内存地址没变

139918090325440
139918090325440 tensor([10.,  1.])


`tensor._version` 检测 tensor 有没有发生 inplace 操作

In [29]:
a = torch.tensor([1.0, 3.0], requires_grad=True)
b = a + 2
print(b._version)

0


In [30]:
loss = (b * b).mean()
b[0] = 1000.0
print(b._version)  # 每次检测到 b 发生 inplace 操作时，_version 就会 +1

1


反向传播时发现 b 已经 inplace 修改了，报错

In [31]:
loss.backward()

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [2]], which is output 0 of AddBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

需要求导的叶子节点要求更严，不用等反向传播，一旦创建后，做 inplace 操作就会报错

In [64]:
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
a.add_(10.) 

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

<br />

## 参考

- [PyTorch 的 Autograd](https://zhuanlan.zhihu.com/p/69294347)